<a href="https://colab.research.google.com/github/wangjb/cwb_open_data/blob/master/fcst_town_future1w.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def retrieve_data(datacode,auth):
  import urllib
  # query string to CWB open data service
  url='https://opendata.cwb.gov.tw/'
  sub_path='fileapi/v1/opendataapi/'
  # instant 10 min precip
  datalimit=''
  dataoffset=''
  dataformat='JSON'
  stationID=''
  datastatus=''

  # create query string
  url_path=url+sub_path+datacode
  query_fields=['Authorization','limit','offset','format','stationID','status']
  query_data=(auth,datalimit,dataoffset,dataformat,stationID,datastatus)
  query_string=dict(zip(query_fields,query_data))

  # request data
  import requests
  r = requests.get(url_path,params=query_string,stream=True)
  jsondata = r.json()

  return jsondata

In [2]:
datacode = 'F-D0047-071'
auth = None
jsondata = retrieve_data(datacode, auth)

In [3]:
jsondata['cwbopendata']['dataset']['datasetInfo']

{'datasetDescription': '臺灣各縣市鄉鎮未來1週逐12小時天氣預報',
 'datasetLanguage': 'zh-TW',
 'issueTime': '2020-08-13T11:00:00+08:00',
 'update': '2020-08-13T12:19:37+08:00',
 'validTime': {'endTime': '2020-08-21T11:00:00+08:00',
  'startTime': '2020-08-13T12:00:00+08:00'}}

In [4]:
cityname = jsondata['cwbopendata']['dataset']['locations']['locationsName']
towndata = jsondata['cwbopendata']['dataset']['locations']['location']

In [5]:
cityname

'新北市'

In [6]:
towndata[0].keys()

dict_keys(['locationName', 'geocode', 'lat', 'lon', 'weatherElement'])

In [7]:
for x in towndata[0]['weatherElement']:
  print(x['description'], x['elementName'])

平均溫度 T
平均露點溫度 Td
平均相對濕度 RH
最高溫度 MaxT
最低溫度 MinT
最高體感溫度 MaxAT
最低體感溫度 MinAT
最大舒適度指數 MaxCI
最小舒適度指數 MinCI
12小時降雨機率 PoP12h
風向 WD
最大風速 WS
天氣現象 Wx
紫外線指數 UVI
天氣預報綜合描述 WeatherDescription


In [8]:
import pandas as pd
from itertools import islice
# process station info
station = pd.DataFrame(list(map(
    lambda x : dict(islice(x.items(), 4)),
      towndata)))
station

,locationName,geocode,lat,lon
0,板橋區,6500100,25.01154,121.450888
1,三重區,6500200,25.062727,121.480099
2,中和區,6500300,25.000438,121.49279
3,永和區,6500400,25.010926,121.511985
4,新莊區,6500500,25.037648,121.442335
5,新店區,6500600,24.969155,121.533397
6,樹林區,6500700,24.991747,121.416592
7,鶯歌區,6500800,24.957831,121.346152
8,三峽區,6500900,24.936061,121.360843
9,淡水區,6501000,25.188869,121.43572


In [9]:
pd.DataFrame.from_dict({ (i,j) : { item['startTime']+'-'+item['endTime'] : 
                  item['elementValue']['value'] if (j<11 or j>13) else item['elementValue'][0]['value']
                          for item in k['time'] }
  for i,v in enumerate(towndata)
  for j,k in enumerate(v['weatherElement'])
},orient='Index')

2020-08-13T12:00:00+08:00-2020-08-13T18:00:00+08:00  ... 2020-08-19T18:00:00+08:00-2020-08-20T06:00:00+08:00
0  0                                                  34   ...                                                 29 
   1                                                  27   ...                                                 25 
   2                                                  66   ...                                                 80 
   3                                                  35   ...                                                 31 
   4                                                  32   ...                                                 28 
...                                                  ...   ...                                                ... 
28 10                                                西北風   ...                                                東南風 
   11                                                  1   ...                                                  3 
   12                                             多雲短暫陣雨   ...                                                 多雲 
   13                                                 10   ...                                                NaN 
   14  多雲短暫陣雨。降雨機率 40%。溫度攝氏28至32度。舒適至悶熱。西北風 風速<= 1級(每...   ...          多雲。溫度攝氏23至27度。舒適。東南風 風速2級(每秒3公尺)。相對濕度90%。 

[435 rows x 14 columns]